# Przygotowanie

In [12]:
import pandas as pd
import numpy as np
import pickle

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline, Pipeline

from nltk.tokenize import TweetTokenizer

In [13]:
from const import PREPROCESSED_TEST_PATH, PREPROCESSED_TRAIN_PATH

# Dane

In [14]:
df_train = pd.read_pickle(PREPROCESSED_TRAIN_PATH)
df_test = pd.read_pickle(PREPROCESSED_TEST_PATH)

In [15]:
Y_train = df_train.target.to_numpy()
Y_test = df_test.target.to_numpy()

In [16]:
X_train = df_train.text.to_numpy()
X_test = df_test.text.to_numpy()

Podobnie jak w przypadku sieci neuronowych musimy zwektoryzować nasz teks. W tym celu możemy wykorzystać __word2vec__. Jednak w tym przykładzie postanowiłem skorzystać z __TfidfVectorizer__, którego niestety z przyczyn wydajnościowych musiałem zamienić na __CountVectorizer__.

In [17]:
# Ten kod jest nieużywane w dalszej części notebooka.
# Pokazuje on jedynie inną metodę konwersji danych do wektora.

tfv = TfidfVectorizer(sublinear_tf=True)
tfv.fit(df_train.text)

X_train_t = tfv.transform(df_train.text)
X_test_t = tfv.transform(df_test.text)

Aby użyć __CountVectorizer__ poprawnie, należy dodać funkcję, która zamienia lity tekst na tokeny. Postanowiłem skorzystać z klasy znajdującej się w __nltk__ - __TweetTokenizer__. Wybrałem ją w oparciu o porównanie znajdujące się https://towardsdatascience.com/an-introduction-to-tweettokenizer-for-processing-tweets-9879389f8fe7

In [18]:
def tokenize(text): 
    tknzr = TweetTokenizer()
    return tknzr.tokenize(text)

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    ngram_range=(1, 1))

In [19]:
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenize at 0x000001F94D9DBE58>,
                vocabulary=None)

Możemy teraz stowrzyć teraz połączyć model z vectorizerem za pomocą pipelinu.

In [20]:
pipeline_svm = make_pipeline(vectorizer, 
                            SVC())

In [21]:
grid_svm = GridSearchCV(pipeline_svm,
                        param_grid = {'svc__C': [0.01, 0.1, 1, 10, 100, 1000]}, 
                        verbose=10,   
                        n_jobs=-1) 

# Uczenie

In [22]:
grid_svm.fit(X_train, Y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 29.6min
[Parallel(n_jobs=-1)]: Done  10 out of  15 | elapsed: 44.0min remaining: 22.0min
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed: 57.7min remaining: 14.4min
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 80.8min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('countvectorizer',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                              

In [23]:
grid_svm.score(X_test, Y_test)

0.7649375

In [25]:
pickle.dump(grid_svm.best_estimator_, open("SVM.model", 'wb'))

In [27]:
grid_svm.best_params_

{'svc__C': 1}